<a href="https://colab.research.google.com/github/dede0702/Assistente-Virtual-RAG-para-Desastres/blob/main/Assistente_Virtual_para_Desastres_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
 1: Instalações e Imports
```



In [4]:
!pip install openai pandas python-dotenv -q



```
2: Imports e Configuração da API Key
```



In [5]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv # Para carregar variáveis de ambiente, se você usar um .env no Colab
import getpass # Para entrada segura da API Key

In [7]:
# --- Configurações Iniciais ---

# Tente carregar de um arquivo .env (opcional, se você fizer upload de um)
# Crie um arquivo .env no seu Colab com a linha:
# OPENAI_API_KEY="sua_chave_aqui"
# E faça o upload para a sessão do Colab.
# Ou, use o input abaixo.
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    print("A API Key da OpenAI (OPENAI_API_KEY) não foi encontrada no ambiente.")
    OPENAI_API_KEY = getpass.getpass("Por favor, insira sua OpenAI API Key e pressione Enter: ")

if not OPENAI_API_KEY:
    print("Erro Crítico: Nenhuma API Key da OpenAI foi fornecida. O programa não pode continuar.")
    # Em um notebook, podemos parar a execução ou levantar um erro
    # raise ValueError("API Key da OpenAI não fornecida.")
    # Por ora, vamos apenas imprimir e deixar o usuário lidar com isso.
else:
    print("API Key da OpenAI carregada com sucesso.")


A API Key da OpenAI (OPENAI_API_KEY) não foi encontrada no ambiente.
Por favor, insira sua OpenAI API Key e pressione Enter: ··········
API Key da OpenAI carregada com sucesso.


In [8]:
# Modelo da OpenAI que você deseja usar
OPENAI_MODEL_ID = "gpt-4o-mini" # Usando o modelo GPT-4o mini

# Inicializar o cliente da API da OpenAI
client = None
if OPENAI_API_KEY:
    try:
        client = OpenAI(api_key=OPENAI_API_KEY)
        print(f"Cliente OpenAI inicializado com sucesso para o modelo: {OPENAI_MODEL_ID}")
    except Exception as e:
        print(f"Erro ao inicializar o cliente da API da OpenAI: {e}")
        client = None # Garante que o cliente não será usado se a inicialização falhar
else:
    print("Cliente OpenAI não pôde ser inicializado pois a API Key não está disponível.")

Cliente OpenAI inicializado com sucesso para o modelo: gpt-4o-mini




```
3: Funções do Sistema RAG (semelhantes ao original)
```



In [9]:
def retrieve_information(query: str, user_profile: str) -> str:
    """
    Simula a recuperação de informações com base na consulta e no perfil do usuário.
    No Colab, esta função permanece conceitualmente a mesma.
    """
    context = ""
    query_lower = query.lower()

    # Lógica de recuperação (mantida do seu código original)
    if "enchente" in query_lower:
        if user_profile == "Vítima":
            context = "Situação de enchente: Procure um local alto e seguro. Evite contato com a água da enchente. Sinalize sua localização para equipes de resgate. Mantenha a calma. Telefones de emergência: Defesa Civil 199, Bombeiros 193."
            if "preso" in query_lower:
                context += " Se estiver preso, não tente atravessar áreas alagadas por conta própria, aguarde o resgate. Se possível, avise sobre sua condição e localização exata."
        elif user_profile == "Morador":
            context = "Alerta de enchente na região: Prepare um kit de emergência (água, comida, medicamentos, lanterna). Desligue a energia elétrica se houver risco de a água atingir sua casa. Mantenha-se informado pelos canais oficiais. Abrigos públicos: Ginásio Municipal, Escola Estadual Central."
        elif user_profile == "Familiar":
            context = "Informações sobre enchentes: Para localizar familiares, entre em contato com a Defesa Civil (199) ou verifique listas em abrigos (Ginásio Municipal, Escola Estadual Central). Evite espalhar boatos, busque informações oficiais."
    elif "incêndio florestal" in query_lower:
        if user_profile == "Vítima":
            context = "Situação de incêndio florestal próximo: Se receber ordem de evacuação, saia imediatamente. Cubra nariz e boca com um pano úmido. Se não puder sair, procure um local aberto e longe da vegetação densa. Ligue para 193 (Bombeiros)."
        elif user_profile == "Morador":
            context = "Alerta de incêndio florestal: Mantenha a vegetação ao redor da casa limpa. Tenha rotas de fuga planejadas. Se avistar fumaça ou focos de incêndio, ligue para 193. Não faça queimadas."
        elif user_profile == "Familiar":
            context = "Busca de informações sobre incêndio: Contate autoridades locais para status da situação e informações sobre áreas afetadas. Verifique notícias de fontes confiáveis."
    else:
        context = "Informações gerais sobre segurança em desastres: Mantenha um kit de emergência, tenha um plano familiar, mantenha-se informado por fontes oficiais e siga as orientações das autoridades."
    return context

def generate_augmented_response(user_query: str, retrieved_context: str, user_profile: str) -> str:
    """
    Gera uma resposta aumentada usando o cliente OpenAI.
    As mensagens de erro são impressas no console do Colab.
    """
    if not client:
        return "Erro: O cliente da API da OpenAI não foi inicializado corretamente. Verifique sua API Key."

    system_prompt = f"""Você é um assistente virtual especializado em orientar pessoas durante desastres naturais.
    Sua missão é fornecer informações claras, precisas, empáticas e acionáveis.
    Você DEVE usar as "Informações Relevantes Recuperadas" para basear sua resposta.
    NÃO invente informações que não foram recuperadas, especialmente números de telefone, endereços específicos ou detalhes factuais não fornecidos no contexto.
    Se as informações recuperadas não forem totalmente suficientes, você pode complementar com conselhos gerais de segurança relevantes para o contexto do desastre mencionado e o perfil do usuário, mas sempre indique claramente quando uma informação é um conselho geral e não parte dos dados recuperados.
    Se a consulta for vaga, peça mais detalhes de forma gentil para poder ajudar melhor.
    Responda diretamente à pergunta do usuário.
    Priorize a segurança e o bem-estar. Seja empático, calmo e direto.
    O perfil do usuário atual é: {user_profile}"""

    user_content_prompt = f"""
    **Consulta do Usuário:**
    "{user_query}"

    **Informações Relevantes Recuperadas (use estas informações para basear sua resposta):**
    "{retrieved_context}"

    Com base no seu papel, no perfil do usuário, na consulta e nas informações recuperadas, forneça a orientação adequada:
    """

    print("\n--- CONSOLE DEBUG: INÍCIO DE generate_augmented_response (OpenAI) ---")
    print(f"CONSOLE DEBUG: Timestamp: {pd.Timestamp.now()}")
    print(f"CONSOLE DEBUG: Usando OPENAI_MODEL_ID: {OPENAI_MODEL_ID}")
    print(f"CONSOLE DEBUG: Comprimento System Prompt: {len(system_prompt)}, User Prompt: {len(user_content_prompt)}")
    print("---------------------------------------------------------")

    try:
        print(f"CONSOLE DEBUG: {pd.Timestamp.now()} - Tentando chamar OpenAI API com modelo de CHAT...")

        response = client.chat.completions.create(
            model=OPENAI_MODEL_ID,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_content_prompt}
            ],
            max_tokens=800,
            temperature=0.4,
            top_p=1.0,
        )
        full_response = response.choices[0].message.content.strip()

        print(f"CONSOLE DEBUG: {pd.Timestamp.now()} - Chamada à API OpenAI (Chat) retornou.")
        print(f"CONSOLE DEBUG: {pd.Timestamp.now()} - Resposta bruta do modelo (primeiros 300 chars): '{full_response[:300]}...'")
        print(f"CONSOLE DEBUG: {pd.Timestamp.now()} - Comprimento total da resposta bruta: {len(full_response)}")
        if response.usage:
            print(f"CONSOLE DEBUG: Uso de tokens: {response.usage}")
        print("--- CONSOLE DEBUG: FIM DE generate_augmented_response (SUCESSO OpenAI) ---\n")
        return full_response

    except Exception as e:
        print("\n!!!!!!!!!! OCORREU UMA EXCEÇÃO AO CHAMAR A API DA OPENAI (CONSOLE LOG) !!!!!!!!!!!")
        error_type_name = type(e).__name__
        print(f"CONSOLE DEBUG: Timestamp da Exceção: {pd.Timestamp.now()}")
        print(f"Tipo da Exceção: {error_type_name}")
        print(f"Mensagem da Exceção: {str(e)}")

        # Tentar imprimir detalhes adicionais do erro, se disponíveis
        if hasattr(e, 'http_status'):
            print(f"HTTP Status: {e.http_status}")
        if hasattr(e, 'code'):
            print(f"OpenAI Error Code: {e.code}")
        if hasattr(e, 'param'):
            print(f"Error Param: {e.param}")
        if hasattr(e, 'body'): # Tentar mostrar o corpo do erro bruto
            print(f"Error Body: {e.body}")
        elif hasattr(e, 'json_body'): # Se o corpo for JSON
             print(f"Error JSON Body: {json.dumps(e.json_body, indent=2)}")

        print("--- CONSOLE DEBUG: FIM DE generate_augmented_response (EXCEÇÃO OpenAI) ---\n")
        return "Desculpe, ocorreu um problema ao tentar gerar sua resposta com a OpenAI. Verifique o console para mais detalhes sobre o erro."




```
4: Interação com o Usuário e Execução Principal
```



In [10]:
def main_colab_interaction():
    print("--- Assistente Virtual para Desastres Naturais (Versão Colab) ---")
    print(f"Utilizando o modelo: {OPENAI_MODEL_ID} (via OpenAI API)")
    print("-" * 60)

    if not client:
        print("\nAVISO: O cliente OpenAI não está pronto. Não será possível gerar respostas do assistente.")
        print("Por favor, verifique sua API Key na Célula 2 e execute-a novamente.")
        return

    # Seleção de Perfil
    user_profile_options = ["Vítima", "Morador", "Familiar"]
    print("\n👤 Selecione seu perfil:")
    for i, option in enumerate(user_profile_options):
        print(f"{i+1}. {option}")

    profile_choice = ""
    while not profile_choice.isdigit() or not (1 <= int(profile_choice) <= len(user_profile_options)):
        profile_choice = input(f"Digite o número do seu perfil (1-{len(user_profile_options)}): ")
        if not profile_choice.isdigit() or not (1 <= int(profile_choice) <= len(user_profile_options)):
            print("Opção inválida. Por favor, tente novamente.")
    selected_profile = user_profile_options[int(profile_choice) - 1]
    print(f"Perfil selecionado: {selected_profile}")
    print("-" * 60)

    # Entrada da Pergunta
    user_query = input("❓ Descreva sua situação ou dúvida:\n(Ex: Estou em uma área de enchente e a água está subindo. O que faço?)\n")
    print("-" * 60)

    if not user_query:
        print("⚠️ Você não digitou uma pergunta. Tente novamente.")
        return

    print("⏳ Processando sua solicitação... Buscando informações e gerando orientação...")

    # 1. Recuperação de Informação (Simulada)
    print("\n--- Contexto Recuperado (Informações de Apoio) ---")
    retrieved_info = retrieve_information(user_query, selected_profile)
    if retrieved_info:
        print(f"DEBUG: Buscando informações para: '{user_query}' (Perfil: {selected_profile})...")
        print(f"```\n{retrieved_info}\n```")
        print(f"DEBUG: Contexto recuperado: {retrieved_info}")
    else:
        print("Nenhum contexto específico foi recuperado para esta consulta (usando informações gerais).")
    print("-" * 60)

    # 2. Geração Aumentada
    print("\n💡 Orientação do Assistente:")
    final_response = generate_augmented_response(user_query, retrieved_info, selected_profile)
    print(final_response)
    print("-" * 60)

    print("\n**Atenção:** Este é um assistente virtual para demonstração e apoio.")
    print("Em situações reais de emergência, **sempre siga as orientações das autoridades locais**")
    print("e contate os serviços de emergência (Bombeiros: 193, Defesa Civil: 199, SAMU: 192).")
    print("-" * 60)



```
5: Executar a interação
```



In [11]:
if __name__ == "__main__": # Garante que só execute se rodado como script principal
    if client: # Só executa a interação se o cliente OpenAI foi inicializado
        main_colab_interaction()
    else:
        print("\nO assistente não pode ser executado porque o cliente OpenAI não foi inicializado.")
        print("Verifique a configuração da sua API Key na Célula 2 e execute-a novamente.")

--- Assistente Virtual para Desastres Naturais (Versão Colab) ---
Utilizando o modelo: gpt-4o-mini (via OpenAI API)
------------------------------------------------------------

👤 Selecione seu perfil:
1. Vítima
2. Morador
3. Familiar
Digite o número do seu perfil (1-3): 2
Perfil selecionado: Morador
------------------------------------------------------------
❓ Descreva sua situação ou dúvida:
(Ex: Estou em uma área de enchente e a água está subindo. O que faço?)
Estou em uma área de enchente e a água está subindo. O que faço?
------------------------------------------------------------
⏳ Processando sua solicitação... Buscando informações e gerando orientação...

--- Contexto Recuperado (Informações de Apoio) ---
DEBUG: Buscando informações para: 'Estou em uma área de enchente e a água está subindo. O que faço?' (Perfil: Morador)...
```
Alerta de enchente na região: Prepare um kit de emergência (água, comida, medicamentos, lanterna). Desligue a energia elétrica se houver risco de a á